# Showcase how to get an SSP template using pyFSPS

This notebook will show you how you get an SSP template using the python binding (*pyFSPS*) to Charly Conroy's *FSPS* package.
For more information about *pyFSPS* see https://dfm.io/python-fsps/current/ for more information about *FSPS* see https://github.com/cconroy20/fsps

Since switching between different supported stellar isochrone and spectral libraries in *pyFSPS* requires (re-) installing *pyFSPS* with specific compiler flags **we do not add a dependence onto *pyFSPS*** and leave the installation of it to the user. See here https://dfm.io/python-fsps/current/installation/ for information on the install process.

Currently Rubix supports to run *pyFSPS* to create an SSP template from scratch -- use the config option `source=rerun_from_scratch`. But note, this is mainly a wrapper around the *pyFSPS* `StellarPopulation()` and `get_spectrum()` functions. If you use the `get_ssp_template()` function from *Rubix* to create the ssp template via *pyFSPS* it will only pass default parameters (`add_neb_emission=True, imf_type=2, zmet=None, tage=0.0, peraa=True`) over to pyFSPS.

While its possible to pass all relevant function parameters to *pyFSPS* through `rubix.spectra.ssp.fsps_grid.retrieve_ssp_data_from_fsps()` or `.write_fsps_data_to_disk()` we recommend to run *pyFSPS* separate from *Rubix* if you want full control over the process.

Since *pyFSPS* runs can take quite some time, this will silently save the ssp template to a *hdf5* file with filename specified under the config entry `filename`. Additionally we support to load pre-existing templates created via *FSPS* using the config option `source=load_from_file` via our standard `HDF5SSPGrid` class.

In [1]:
# NBVAL_SKIP
from rubix.spectra.ssp.factory import get_ssp_template
ssp = get_ssp_template("FSPS")
ssp

2024-11-14 11:25:42,131 - rubix - INFO - 
   ___  __  _____  _____  __
  / _ \/ / / / _ )/  _/ |/_/
 / , _/ /_/ / _  |/ /_>  <
/_/|_|\____/____/___/_/|_|




2024-11-14 11:25:42,132 - rubix - INFO - Rubix version: 0.0.post101+gda5b92f.d20241101


2024-11-14 11:25:42,132 - rubix - WARNING - python-fsps is not installed. Please install it to use this function. Install using pip install fsps and check the installation page: https://dfm.io/python-fsps/current/installation/ for more details. Especially, make sure to set all necessary environment variables.


2024-11-14 11:25:42,134 - rubix - INFO - Running fsps to generate SSP templates. This may take a while.


AssertionError: Must have python-fsps installed to use this function

In [ ]:
# NBVAL_SKIP
ssp.age.shape

In [ ]:
# NBVAL_SKIP
ssp.metallicity.shape

In [ ]:
# NBVAL_SKIP
ssp.wavelength.shape

In [ ]:
# NBVAL_SKIP
ssp.flux.shape

In [ ]:
# NBVAL_SKIP
import os
from rubix.paths import TEMPLATE_PATH
os.path.exists(os.path.join(TEMPLATE_PATH, config['file_name']))

## Let's plot some of the spectra

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# NBVAL_SKIP
plt.plot(ssp.wavelength,ssp.flux[0][0])
plt.xlabel(r'$\lambda$ [%s]'%config["fields"]["wavelength"]["units"])
plt.ylabel(r'Flux [%s]'%config["fields"]["flux"]["units"])
#plt.yscale("log")
plt.xlim(0,10000)

In [ ]:
# NBVAL_SKIP
plt.plot(ssp.wavelength,ssp.flux[-1][-1])
plt.xlabel(r'$\lambda$ [%s]'%config["fields"]["wavelength"]["units"])
plt.ylabel(r'Flux [%s]'%config["fields"]["flux"]["units"])
#plt.yscale("log")
plt.xlim(0,10000)

In [ ]:
# NBVAL_SKIP
for i in range(len(ssp.metallicity)):
    plt.plot(ssp.wavelength,ssp.flux[i][0], label=r'Z=%0.3f'%ssp.metallicity[i])
plt.xlabel(r'$\lambda$ [%s]'%config["fields"]["wavelength"]["units"])
plt.ylabel(r'Flux [%s]'%config["fields"]["flux"]["units"])
#plt.yscale("log")
plt.xlim(0,10000)
plt.legend()

In [ ]:
# NBVAL_SKIP
ages = np.linspace(0,len(ssp.age),10)
for age in ages:
    plt.plot(ssp.wavelength,ssp.flux[0][int(age)], label='%.2f %s'%(ssp.age[int(age)], config["fields"]["age"]["units"]))
plt.xlabel(r'$\lambda$ [%s]'%config["fields"]["wavelength"]["units"])
plt.ylabel(r'Flux [%s]'%config["fields"]["flux"]["units"])
#plt.yscale("log")
plt.xlim(0,5000)
plt.legend()

## Reload the created FSPS template

This is only for tutorial purposes as you would need to run fsps like above first to have a pre-existing template...

In [ ]:
from rubix import config as rubix_config
rubix_config["ssp"]["templates"]["FSPS"]["source"] = "load_from_file"

In [ ]:
# NBVAL_SKIP
ssp2 = get_ssp_template("FSPS")
ssp2

In [ ]:
# NBVAL_SKIP
ssp.wavelength.shape == ssp2.wavelength.shape